## Load in the geneset files
updated version of Geneset files can be downloade from [here](http://download.baderlab.org/EM_Genesets/current_release/) 

Ideally we would be able to select the species and identifier type.  Once the user selects these then the list of currently available files could come up in a drop down or something similar.

In [1]:
download.file(
    "http://download.baderlab.org/EM_Genesets/current_release/Human/symbol/Human_GOBP_AllPathways_no_GO_iea_October_01_2016_symbol.gmt",
    destfile="./Human_GOBP_AllPathways_no_GO_iea_October_01_2016_symbol.gmt"
)

In [2]:
library(GSA)
capture.output(
    genesets <- GSA.read.gmt("Human_GOBP_AllPathways_no_GO_iea_October_01_2016_symbol.gmt"),
    file="./gsea_load_output.txt"
   ) 

## Define the list of genes you are interested in seeing enrichments for

Ideally I would like it as a text box where the user just enters their genes of interest

In [3]:
mesenchymal_genes <- read.table( "./data/mesenvsimmuno_mesenonly_RNAseq_gprofiler.txt", header = FALSE, 
                                sep = "\t", quote="\"",  stringsAsFactors = FALSE)
mesenchymal_genes <- as.vector(t(mesenchymal_genes))

In [4]:
head(mesenchymal_genes)

[1] "BGN"     "ANTXR1"  "FZD1"    "COL16A1" "GPC6"    "TSPAN11"

## Calculate the universe
Either:
* the universe can be supplied by the user (for example the complete set of genes/proteins identified in the experiment regardless of whether it was significant or not) or 
* we can calculate the universe from the gmt file (universe = any gene with an annotation) 

In [5]:
unique_genesets <- genesets$geneset.names
universe_genes <- c()
for(i in 2:length(unique_genesets)){
  #get the current geneset
  current_geneset <- unlist(genesets$genesets[i])
  current_geneset <- current_geneset[which(current_geneset != "")]
    universe_genes <- union(universe_genes, current_geneset)
    }

In [6]:
length(universe_genes)

[1] 14810

## Calculate enrichments for set of genes
Go through each geneset in the geneset file and test the group of genes for enrichment in that geneset using fisher exact test.  

In [101]:
#go through all the genesets
#skip the first line as it is just the header
unique_genesets <- genesets$geneset.names
inlist <- length(mesenchymal_genes)
enrichments_raw <- c()
for(i in 2:length(unique_genesets)){
  #get the current geneset
  current_geneset <- unlist(genesets$genesets[i])
  current_geneset <- current_geneset[which(current_geneset != "")]
  
    #number of genes from set that overlap current geneset
    ings_inlist <- length(intersect(current_geneset, mesenchymal_genes))
    
    #number of genes in geneset
    ings <- length(current_geneset)
    
    #number of genes in geneset and in universe
    ings_inuniverse <- length(intersect(current_geneset,universe_genes))
    
    notings <- inlist - length(ings_inlist)
    notinuniverse <- length(universe_genes) - ings_inuniverse
    
    #calculate fisher exact statistic
    current_pvalue <- fisher.test(matrix(c(ings_inlist, ings_inuniverse, notings, notinuniverse), 2, 2), 
                       alternative="greater")$p.value
    
  enrichments_raw <- rbind(enrichments_raw,c(ings_inlist, ings_inuniverse, notings, notinuniverse, current_pvalue,
                                             paste0(intersect(current_geneset, mesenchymal_genes),collapse=",")))

}

row.names(enrichments_raw) <- unique_genesets[2:length(unique_genesets)]


## Correct for multiple hypothesis testing

In [102]:
#correct the p-values
enriched_pvalue_adjusted <- p.adjust(enrichments_raw[,5], "BH")
enrichments_raw <- cbind(enrichments_raw,enriched_pvalue_adjusted)

## Create output file

In [118]:
# gProfileR returns corrected p-values only.  Set p-value to corrected p-value
mesenchymal_em_results <- data.frame(name=row.names(enrichments_raw),
                                     description=row.names(enrichments_raw),
                                pvalue=as.numeric(enrichments_raw[,5]),
                                    qvalue= as.numeric(enrichments_raw[,7]),
                                     phenotype=1,genes=enrichments_raw[,6],stringsAsFactors = FALSE)

#colnames(mesenchymal_em_results) <- c("Name","Description", "pvalue","qvalue","phenotype","genes")

write.table(mesenchymal_em_results,"fisherexcat_results_mesenonly_.txt",col.name=TRUE,sep="\t",row.names=FALSE,quote=FALSE)


## Build an enrichment Map

In [9]:
library(RJSONIO)

library(httr)
# Basic settings
port.number = 1234
base.url = paste("http://localhost:", toString(port.number), "/v1", sep="")

print(base.url)

version.url = paste(base.url, "version", sep="/")
cytoscape.version = GET(version.url)
cy.version = fromJSON(rawToChar(cytoscape.version$content))
print(cy.version)


[1] "http://localhost:1234/v1"
      apiVersion cytoscapeVersion 
            "v1"          "3.4.0" 


## Define Thresholds

Ideally the user will be able to change the below thresholds and variables

In [10]:
pvalue_thresh <- 0.01
qvalue_thresh <- 0.05
similarity_thresh <- 0.25
similiarity <- "JACCARD"

In [11]:
# to create an Enrichment map we need to specify
# analysisType = generic
# 
enrichmentmap.url <- paste(base.url, "commands","enrichmentmap","build", sep="/") 

path_to_file="/Users/risserlin/Dropbox (Bader Lab)/Ruth Isserlin's files/Enrichment_Analyses/EM-tutorials-docker/notebooks/"
#on windows change the \ to / in order for the pathname to be interpretted correctly.
#path_to_file="C:/Users/zaphod/Ruth_dropbox/Dropbox (Bader Lab)/Ruth Isserlin's files/Enrichment_Analyses/Jupyter_Notebooks/notebooks/data/"

enr_file = paste(path_to_file,"fisherexcat_results_mesenonly_.txt",sep="")

em_params <- list(analysisType = "generic",enrichmentsDataset1 = enr_file,
                  pvalue=pvalue_thresh,qvalue=qvalue_thresh,
                  #expressionDataset1 = exp_file, 
                  similaritycutoff=similarity_thresh,coeffecients=similiarity)

response <- GET(url=enrichmentmap.url, query=em_params)

In [12]:
 content(response, "text", encoding = "ISO-8859-1")

[1] "<p style=\"color:black;margin-left:10px;margin-top:0px;margin-bottom:5px;\">Finished</p>\n"

## Get the image of the created network so you can see the results

Using cyrest I am able to get a png of the network but the network ID is hardcoded.  Need a way to get the network id of the last created network.

In [20]:
url_png <- "http://localhost:1234/v1/networks/1872/views/first.png/"
response <- GET(url=url_png)
bin <-  content(response, "raw")
writeBin(bin, "output.png")

<img src="output.png">